## 1.生成数据集

In [12]:
from mxnet import autograd, nd
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale = 1, shape = (num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale = 0.01, shape = labels.shape)

## 2.读取数据

In [13]:
from mxnet.gluon import data as gdata
batch_size = 10

# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)

# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle = True)

In [14]:
for X, y in data_iter:
    print(X, y)
    break


[[ 1.2151128  -0.29093367]
 [ 0.91662556  0.30858725]
 [ 0.40402216  0.03693812]
 [ 0.8054629   0.10958023]
 [ 0.48518756 -0.9668033 ]
 [-0.40909502  0.1813641 ]
 [ 0.69494295 -0.92470044]
 [ 0.22467583 -0.39145193]
 [-0.41282937  0.2846847 ]
 [ 0.64080966 -0.74297047]]
<NDArray 10x2 @cpu(0)> 
[7.625295  4.982805  4.882336  5.4455385 8.456818  2.7704165 8.751182
 5.970328  2.4167206 8.0292635]
<NDArray 10 @cpu(0)>


## 3.定义模型

In [15]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

## 4.初始化模型参数

In [16]:
from mxnet import init
net.initialize(init.Normal(sigma = 0.01))

## 5.定义损失函数

In [17]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss() # 平方损失又称L2范数损失

## 6.定义优化算法

In [18]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## 7.训练模型

In [19]:
num_epochs =3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.054620
epoch 2, loss: 0.000242
epoch 3, loss: 0.000050


In [20]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.9998107 -3.399565 ]]
 <NDArray 1x2 @cpu(0)>)

In [21]:
true_b, dense.bias.data()

(4.2, 
 [4.1987643]
 <NDArray 1 @cpu(0)>)